#### Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV,\
    cross_val_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report, plot_roc_curve
from sklearn.pipeline import Pipeline
pd.set_option('display.max_columns',None)
%matplotlib inline
sns.set_theme()

In [4]:
from os.path import dirname, join as pjoin
import scipy.io as sio

# Data

## Images MAT File

In [5]:
mat = sio.loadmat('/Users/Mike/Flatiron_020121/datasets/image_datasets/cars/cars_annos.mat')# load mat-file


In [11]:
mat['annotations']

array([[(array(['car_ims/000001.jpg'], dtype='<U18'), array([[112]], dtype=uint8), array([[7]], dtype=uint8), array([[853]], dtype=uint16), array([[717]], dtype=uint16), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
        (array(['car_ims/000002.jpg'], dtype='<U18'), array([[48]], dtype=uint8), array([[24]], dtype=uint8), array([[441]], dtype=uint16), array([[202]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
        (array(['car_ims/000003.jpg'], dtype='<U18'), array([[7]], dtype=uint8), array([[4]], dtype=uint8), array([[277]], dtype=uint16), array([[180]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
        ...,
        (array(['car_ims/016183.jpg'], dtype='<U18'), array([[25]], dtype=uint8), array([[32]], dtype=uint8), array([[587]], dtype=uint16), array([[359]], dtype=uint16), array([[196]], dtype=uint8), array([[1]], dtype=uint8)),
        (array(['car_ims/016184.jpg'], dtype='<U18'), array([[56]], dtype=uint8), array([

In [63]:
mat['class_names']

array([[array(['AM General Hummer SUV 2000'], dtype='<U26'),
        array(['Acura RL Sedan 2012'], dtype='<U19'),
        array(['Acura TL Sedan 2012'], dtype='<U19'),
        array(['Acura TL Type-S 2008'], dtype='<U20'),
        array(['Acura TSX Sedan 2012'], dtype='<U20'),
        array(['Acura Integra Type R 2001'], dtype='<U25'),
        array(['Acura ZDX Hatchback 2012'], dtype='<U24'),
        array(['Aston Martin V8 Vantage Convertible 2012'], dtype='<U40'),
        array(['Aston Martin V8 Vantage Coupe 2012'], dtype='<U34'),
        array(['Aston Martin Virage Convertible 2012'], dtype='<U36'),
        array(['Aston Martin Virage Coupe 2012'], dtype='<U30'),
        array(['Audi RS 4 Convertible 2008'], dtype='<U26'),
        array(['Audi A5 Coupe 2012'], dtype='<U18'),
        array(['Audi TTS Coupe 2012'], dtype='<U19'),
        array(['Audi R8 Coupe 2012'], dtype='<U18'),
        array(['Audi V8 Sedan 1994'], dtype='<U18'),
        array(['Audi 100 Sedan 1994'], dtype='<U

In [36]:
mat['annotations'][0]

array([(array(['car_ims/000001.jpg'], dtype='<U18'), array([[112]], dtype=uint8), array([[7]], dtype=uint8), array([[853]], dtype=uint16), array([[717]], dtype=uint16), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
       (array(['car_ims/000002.jpg'], dtype='<U18'), array([[48]], dtype=uint8), array([[24]], dtype=uint8), array([[441]], dtype=uint16), array([[202]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
       (array(['car_ims/000003.jpg'], dtype='<U18'), array([[7]], dtype=uint8), array([[4]], dtype=uint8), array([[277]], dtype=uint16), array([[180]], dtype=uint8), array([[1]], dtype=uint8), array([[0]], dtype=uint8)),
       ...,
       (array(['car_ims/016183.jpg'], dtype='<U18'), array([[25]], dtype=uint8), array([[32]], dtype=uint8), array([[587]], dtype=uint16), array([[359]], dtype=uint16), array([[196]], dtype=uint8), array([[1]], dtype=uint8)),
       (array(['car_ims/016184.jpg'], dtype='<U18'), array([[56]], dtype=uint8), array([[60]],

In [60]:
mat['annotations'][0][-1]

(array(['car_ims/016185.jpg'], dtype='<U18'), array([[1]], dtype=uint8), array([[1]], dtype=uint8), array([[200]], dtype=uint8), array([[131]], dtype=uint8), array([[196]], dtype=uint8), array([[1]], dtype=uint8))

In [35]:
mat['annotations'][0][0]

(array(['car_ims/000001.jpg'], dtype='<U18'), array([[112]], dtype=uint8), array([[7]], dtype=uint8), array([[853]], dtype=uint16), array([[717]], dtype=uint16), array([[1]], dtype=uint8), array([[0]], dtype=uint8))

In [34]:
mat['annotations'][0][0][2]

array([[7]], dtype=uint8)

In [33]:
mat['annotations'][0][0][2][0]

array([7], dtype=uint8)

In [49]:
mat['annotations'][0][0][1][0][0]

112

In [42]:
mat['annotations'][0][0][2][0]

array([7], dtype=uint8)

In [ ]:
def mat_to_df(mat_file):
    for i in mat_file['annotations'][0]:
        df['image'] = i[0][0]
        df['relative_im_path'] = i[1][0][0]
        df['x1'] = i[2][0][0]
        df['y1'] = i[3][0][0]
        df['x2'] = i[4][0][0]
        df['y2'] = i[5][0][0]
        df['class'] = i[6][0][0]
        
        
    
    

In [65]:
len(mat['annotations'][0])

16185

In [ ]:
# mat = sio.loadmat('/Users/Mike/Flatiron_020121/datasets/image_datasets/cars/cars_annos.mat')# load mat-file
# mdata = mat['myVar']  # variable in mat file 
# ndata = {n: mdata[n][0,0] for n in mdata.dtype.names}
# Columns = [n for n, v in ndata.items() if v.size == 1]
# d=dict((c, ndata[c][0]) for c in Columns)
# df=pd.DataFrame.from_dict(d)
# display(df)

In [70]:
mat['class_names'][0][2][0]

'Acura TL Sedan 2012'

In [217]:
car_list = []
for i in mat['class_names'][0]:
    car_list.append(i[0])

Now that we have a list of cars, let's separate `make`, `model`, and `year` in to separate keys/value pairs of a dictionary.

In [218]:
car_mmy = []
for i in car_list:
    sep = i.split(' ')
    make = sep[0]
    model = sep[1:-1]
    year = sep[-1]
    car_mmy.append({'make': make, 'model': model, 'year': year})
car_mmy[:5]

[{'make': 'AM', 'model': ['General', 'Hummer', 'SUV'], 'year': '2000'},
 {'make': 'Acura', 'model': ['RL', 'Sedan'], 'year': '2012'},
 {'make': 'Acura', 'model': ['TL', 'Sedan'], 'year': '2012'},
 {'make': 'Acura', 'model': ['TL', 'Type-S'], 'year': '2008'},
 {'make': 'Acura', 'model': ['TSX', 'Sedan'], 'year': '2012'}]

In [219]:
for i in car_mmy:
    if i['make'] == 'AM' or i['make'] == 'Aston' or i['make'] == 'Land' or i['make'] == 'MINI':
        make = i['make']
        model = i['model'][0]
        i['make'] = f'{make}'+' '+f'{model}'
        i['model'].pop(0)
car_mmy[45:52]

[{'make': 'Bugatti', 'model': ['Veyron', '16.4', 'Coupe'], 'year': '2009'},
 {'make': 'Buick', 'model': ['Regal', 'GS'], 'year': '2012'},
 {'make': 'Buick', 'model': ['Rainier', 'SUV'], 'year': '2007'},
 {'make': 'Buick', 'model': ['Verano', 'Sedan'], 'year': '2012'},
 {'make': 'Buick', 'model': ['Enclave', 'SUV'], 'year': '2012'},
 {'make': 'Cadillac', 'model': ['CTS-V', 'Sedan'], 'year': '2012'},
 {'make': 'Cadillac', 'model': ['SRX', 'SUV'], 'year': '2012'}]

In [220]:
print(set([i['model'][-1] for i in car_mmy])) # checking unique car body styles

{'SS', 'ZR1', 'IPL', 'Sedan', 'Z06', 'GS', 'SuperCab', 'SRT8', 'Superleggera', 'R', 'Van', 'SRT-8', 'Minivan', 'Wagon', 'Cab', 'Hatchback', 'Type-S', 'XKR', 'SUV', 'Convertible', 'Coupe', 'Abarth'}


In [221]:
car_mmy[0]['model'][-1] # body style

'SUV'

In [222]:
for i in car_mmy:
    if i['model'][-1] == 'Sedan' \
    or i['model'][-1] == 'SuperCab' \
    or i['model'][-1] == 'Minivan' \
    or i['model'][-1] == 'Wagon' \
    or i['model'][-1] == 'Hatchback' \
    or i['model'][-1] == 'SUV' \
    or i['model'][-1] == 'Coupe':
        bodystyle = i['model'][-1]
        i['body_style'] = bodystyle
        i['model'].pop(-1)
    elif i['model'][-1] == 'Convertible':
        bodystyle = i['model'][-1]
        i['body_style'] = bodystyle
        if i['model'][-2] == 'Conv.':
            i['model'].pop(-2)
        i['model'].pop(-1)
    elif i['model'][-1] == 'Cab':
        if i['model'][-2] == 'Crew' or 'Extended' or 'Regular' or 'Quad' or 'Club':
            bigram2 = i['model'][-1]
            bigram1 = i['model'][-2]
            i['body_style'] = f'{bigram1}'+' '+f'{bigram2}'
            i['model'].pop(-2)
            i['model'].pop(-1)
    elif i['model'][-1] == 'Van':
        if i['model'][-2] == 'Cargo' or 'Passenger':
            bigram2 = i['model'][-1]
            bigram1 = i['model'][-2]
            i['body_style'] = f'{bigram1}'+' '+f'{bigram2}'
            i['model'].pop(-2)
            i['model'].pop(-1)

In [223]:
def list_to_string(s): 
    """
    This function takes a list and converts its contents in to a single string object.
    """
    # initializing an empty string
    string_bag = ''
    # adding list items to string
    for x in s: 
        string_bag += x 
        string_bag += ' '
    return string_bag.strip(' ')

In [224]:
for i in car_mmy:
    i['model'] = list_to_string(i['model'])

In [179]:
# for i in car_mmy:
#     holding_list = []
#     string_bag = ''
#     for x in i['model']:
#         holding_list.extend(x)
#     for x in holding_list:
#         string_bag.join(f'{x} ')
#     string_bag.strip(' ')
#     i['model'] = string_bag
#     print(holding_list)


['H', 'u', 'm', 'm', 'e', 'r']
['R', 'L']
['T', 'L']
['T', 'L', 'T', 'y', 'p', 'e', '-', 'S']
['T', 'S', 'X']
['I', 'n', 't', 'e', 'g', 'r', 'a', 'T', 'y', 'p', 'e', 'R']
['Z', 'D', 'X']
['V', '8', 'V', 'a', 'n', 't', 'a', 'g', 'e']
['V', '8', 'V', 'a', 'n', 't', 'a', 'g', 'e']
['V', 'i', 'r', 'a', 'g', 'e']
['V', 'i', 'r', 'a', 'g', 'e']
['R', 'S', '4']
['A', '5']
['T', 'T', 'S']
['R', '8']
['V', '8']
['1', '0', '0']
['1', '0', '0']
['T', 'T']
['S', '6']
['S', '5']
['S', '5']
['S', '4']
['S', '4']
['T', 'T', 'R', 'S']
['A', 'c', 't', 'i', 'v', 'e', 'H', 'y', 'b', 'r', 'i', 'd', '5']
['1', 'S', 'e', 'r', 'i', 'e', 's']
['1', 'S', 'e', 'r', 'i', 'e', 's']
['3', 'S', 'e', 'r', 'i', 'e', 's']
['3', 'S', 'e', 'r', 'i', 'e', 's']
['6', 'S', 'e', 'r', 'i', 'e', 's']
['X', '5']
['X', '6']
['M', '3']
['M', '5']
['M', '6']
['X', '3']
['Z', '4']
['C', 'o', 'n', 't', 'i', 'n', 'e', 'n', 't', 'a', 'l', 'S', 'u', 'p', 'e', 'r', 's', 'p', 'o', 'r', 't', 's']
['A', 'r', 'n', 'a', 'g', 'e']
['M', 'u',

In [225]:
car_mmy

[{'make': 'AM General',
  'model': 'Hummer',
  'year': '2000',
  'body_style': 'SUV'},
 {'make': 'Acura', 'model': 'RL', 'year': '2012', 'body_style': 'Sedan'},
 {'make': 'Acura', 'model': 'TL', 'year': '2012', 'body_style': 'Sedan'},
 {'make': 'Acura', 'model': 'TL Type-S', 'year': '2008'},
 {'make': 'Acura', 'model': 'TSX', 'year': '2012', 'body_style': 'Sedan'},
 {'make': 'Acura', 'model': 'Integra Type R', 'year': '2001'},
 {'make': 'Acura', 'model': 'ZDX', 'year': '2012', 'body_style': 'Hatchback'},
 {'make': 'Aston Martin',
  'model': 'V8 Vantage',
  'year': '2012',
  'body_style': 'Convertible'},
 {'make': 'Aston Martin',
  'model': 'V8 Vantage',
  'year': '2012',
  'body_style': 'Coupe'},
 {'make': 'Aston Martin',
  'model': 'Virage',
  'year': '2012',
  'body_style': 'Convertible'},
 {'make': 'Aston Martin',
  'model': 'Virage',
  'year': '2012',
  'body_style': 'Coupe'},
 {'make': 'Audi',
  'model': 'RS 4',
  'year': '2008',
  'body_style': 'Convertible'},
 {'make': 'Audi', '

In [ ]:
#   """
#   MULTIWORD BODY STYLES BELOW
#   """     
#     {'make': 'Bentley', 'model': ['Continental', 'Supersports', 'Conv.', 'Convertible'], 'year': '2012'}
#     {'make': 'Cadillac', 'model': ['Escalade', 'EXT', 'Crew', 'Cab'], 'year': '2007'}
#     {'make': 'Chevrolet', 'model': ['Silverado', '1500', 'Hybrid', 'Crew', 'Cab'], 'year': '2012'}
#     {'make': 'Chevrolet', 'model': ['Express', 'Cargo', 'Van'], 'year': '2007'}
#     {'make': 'Chevrolet', 'model': ['Avalanche', 'Crew', 'Cab'], 'year': '2012'}
#     {'make': 'Chevrolet', 'model': ['Silverado', '2500HD', 'Regular', 'Cab'], 'year': '2012'}
#     {'make': 'Chevrolet', 'model': ['Silverado', '1500', 'Classic', 'Extended', 'Cab'], 'year': '2007'}
#     {'make': 'Chevrolet', 'model': ['Silverado', '1500', 'Extended', 'Cab'], 'year': '2012'}
#     {'make': 'Chevrolet', 'model': ['Silverado', '1500', 'Regular', 'Cab'], 'year': '2012'}
#     {'make': 'Dodge', 'model': ['Ram', 'Pickup', '3500', 'Crew', 'Cab'], 'year': '2010'}
#     {'make': 'Dodge', 'model': ['Ram', 'Pickup', '3500', 'Quad', 'Cab'], 'year': '2009'}
#     {'make': 'Dodge', 'model': ['Sprinter', 'Cargo', 'Van'], 'year': '2009'}
#     {'make': 'Dodge', 'model': ['Dakota', 'Crew', 'Cab'], 'year': '2010'}
#     {'make': 'Dodge', 'model': ['Dakota', 'Club', 'Cab'], 'year': '2007'}
#     {'make': 'Ford', 'model': ['F-450', 'Super', 'Duty', 'Crew', 'Cab'], 'year': '2012'}
#     {'make': 'Ford', 'model': ['F-150', 'Regular', 'Cab'], 'year': '2012'}
#     {'make': 'Ford', 'model': ['F-150', 'Regular', 'Cab'], 'year': '2007'}
#     {'make': 'GMC', 'model': ['Canyon', 'Extended', 'Cab'], 'year': '2012'}
#     {'make': 'HUMMER', 'model': ['H3T', 'Crew', 'Cab'], 'year': '2010'}
#     {'make': 'HUMMER', 'model': ['H2', 'SUT', 'Crew', 'Cab'], 'year': '2009'}
#     {'make': 'Nissan', 'model': ['NV', 'Passenger', 'Van'], 'year': '2012'}
#     {'make': 'Ram', 'model': ['C/V', 'Cargo', 'Van', 'Minivan'], 'year': '2012'}     

#    """
#    HYBRID LISTED IN NAME BELOW
#    """
#     {'make': 'Chevrolet', 'model': ['Tahoe', 'Hybrid', 'SUV'], 'year': '2012'}
#     {'make': 'Chevrolet', 'model': ['Malibu', 'Hybrid', 'Sedan'], 'year': '2010'}
#     {'make': 'GMC', 'model': ['Yukon', 'Hybrid', 'SUV'], 'year': '2012'}
#     {'make': 'Hyundai', 'model': ['Sonata', 'Hybrid', 'Sedan'], 'year': '2012'}

In [ ]:
# nba['concat'] = nba['Year'].astype(str) + nba['Tm']
# champs['concat'] = champs['Year'].astype(str) + champs['Tm']
# champs['runner_up'] = champs['Year'].astype(str) + champs['Runner_Up_Code']
# nba['champ'] = nba['concat'].isin(champs['concat']).astype(int)
# nba['runner_up'] = nba['concat'].isin(champs['runner_up']).astype(int)
# nba.head()

## Fuel Economy Dataset

In [78]:
big_mpg_df[big_mpg_df.phevBlended == True].head(3)

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
24687,4.744612,3.469579,0.0,1.5,51,50.5282,90,90.2714,0.35,26.0,0.3170,133,-1,0.0,133.0,50,49.7021,95,95.3887,29.0,0.1907,0.2880,4.0,1.8,Front-Wheel Drive,85,PHEV,-1,850,0,Regular Gas and Electricity,Regular Gasoline,-1,-1,49,48.7284,102,102.4898,0.0,33.0,0.2510,22,94,32484,0,0,Toyota,Prius Plug-in Hybrid,Y,True,0,0,0,0.0,12.15,0.0,9.1,Automatic (variable gear ratios),70.9,128.9591,68.4,146.4140,Midsize Cars,2012,4000,NaN,NaN,NaN,NaN,Plug-in Hybrid,Electricity,11,18 kW,TYX,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jun 02 00:00:00 EDT 2020,Y,59,56,58
25629,4.737144,3.469579,0.0,1.5,51,50.5282,90,90.2714,0.35,26.0,0.3200,133,-1,0.0,133.0,50,49.7021,95,95.3887,29.0,0.1907,0.2900,4.0,1.8,Front-Wheel Drive,43,PHEV,10,850,0,Regular Gas and Electricity,Regular Gasoline,10,-1,49,48.7284,102,102.4898,0.0,33.0,0.2500,22,94,33335,0,0,Toyota,Prius Plug-in Hybrid,Y,True,0,0,0,0.0,12.15,0.0,9.1,Automatic (variable gear ratios),70.9,128.9591,68.4,146.4140,Midsize Cars,2013,4000,NaN,NaN,NaN,NaN,Plug-in Hybrid,Electricity,11,18 kW,TYX,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Mon Sep 26 00:00:00 EDT 2016,Y,59,56,58
25630,4.878704,3.745568,0.0,2.5,40,40.4488,95,94.5114,0.00,34.0,0.4781,129,-1,0.0,129.0,38,38.4528,88,87.5757,37.0,0.0000,0.4506,4.0,2.0,Front-Wheel Drive,149,PHEV,10,1100,0,Regular Gas and Electricity,Regular Gasoline,10,-1,36,36.2655,81,81.4490,0.0,42.0,0.4144,0,0,33336,0,19,Ford,C-MAX Energi Plug-in Hybrid,Y,True,0,100,0,0.0,21.40,0.0,17.4,Automatic (variable gear ratios),55.0,135.0163,51.4,115.6155,Midsize Cars,2013,3250,NaN,NaN,NaN,NaN,Plug-in Hybrid,Electricity,20,68 kW,FMX,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Mon Sep 26 00:00:00 EDT 2016,Y,55,46,51


In [79]:
# big_mpg_df['make', 'model', 'year', 'mpgData', 'UCity', 'UCityA', 'UHighway', 'UHighwayA', 'VClass', 'phevBlended', 'feScore', 'fuelType2', 'evMotor', 'comb08', 'comb08U', 'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'phevCity', 'phevHwy', 'phevComb', 'createdOn', 'modifiedOn']

KeyError: ('make', 'model', 'year', 'mpgData', 'UCity', 'UCityA', 'UHighway', 'UHighwayA', 'VClass', 'phevBlended', 'feScore', 'fuelType2', 'evMotor', 'comb08', 'comb08U', 'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'phevCity', 'phevHwy', 'phevComb', 'createdOn', 'modifiedOn')

In [226]:
big_mpg_df

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,2050,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-2250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,29.964545,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,3900,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-11500,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,12.207778,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,1600,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,0,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,29.964545,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,3900,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-11500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,2750,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-5750,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43491,14.982273,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1950,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,0.0,0.0,0.0,0,0,9995,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Automatic 4-spd,24.0000,0.0,37.0000,0.0,Compact Cars,1993,-1750,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
43492,14.330870,0.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,386.391304,23,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,1850,0,Regular,Regular Gasoline,-1,-1,28,0.0,0,0.0,0.0,0.0,0.0,0,0,9996,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,25.0000,0.0,39.0000,0.0,Compact

In [228]:
len(big_mpg_df.columns)

83

In [80]:
big_mpg_df = pd.read_csv('/Users/Mike/Flatiron_020121/p4-nn/data/vehicles.csv')
mpg_df = pd.DataFrame(data=big_mpg_df, columns=['make', 'model', 'year', 
                                                'mpgData', 'UCity', 'UCityA', 'UHighway', 'UHighwayA', 
                                                'VClass', 'phevBlended', 'feScore', 'fuelType2', 'evMotor', 
                                                'comb08', 'comb08U', 'combA08', 'combA08U', 'combE', 
                                                'combinedCD', 'combinedUF', 
                                                'phevCity', 'phevHwy', 'phevComb', 
                                                'createdOn', 'modifiedOn'])

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [97]:
mpg_df.iloc[:2]

,make,model,year,mpgData,UCity,UCityA,UHighway,UHighwayA,VClass,phevBlended,feScore,fuelType2,evMotor,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,phevCity,phevHwy,phevComb,createdOn,modifiedOn
0,Alfa Romeo,Spider Veloce 2000,1985,Y,23.3333,0.0,35.0,0.0,Two Seaters,False,-1,NaN,NaN,21,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013
1,Ferrari,Testarossa,1985,N,11.0000,0.0,19.0,0.0,Two Seaters,False,-1,NaN,NaN,11,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013


In [102]:
mpg_df.iterrows()

,make,model,year,mpgData,UCity,UCityA,UHighway,UHighwayA,VClass,phevBlended,feScore,fuelType2,evMotor,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,phevCity,phevHwy,phevComb,createdOn,modifiedOn
0,Alfa Romeo,Spider Veloce 2000,1985,Y,23.3333,0.0,35.0000,0.0,Two Seaters,False,-1,NaN,NaN,21,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013
1,Ferrari,Testarossa,1985,N,11.0000,0.0,19.0000,0.0,Two Seaters,False,-1,NaN,NaN,11,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013
2,Dodge,Charger,1985,Y,29.0000,0.0,47.0000,0.0,Subcompact Cars,False,-1,NaN,NaN,27,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013
3,Dodge,B150/B250 Wagon 2WD,1985,N,12.2222,0.0,16.6667,0.0,Vans,False,-1,NaN,NaN,11,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013
4,Subaru,Legacy AWD Turbo,1993,N,21.0000,0.0,32.0000,0.0,Compact Cars,False,-1,NaN,NaN,19,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43491,Subaru,Legacy,1993,N,24.0000,0.0,37.0000,0.0,Compact Cars,False,-1,NaN,NaN,22,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013
43492,Subaru,Legacy,1993,N,25.0000,0.0,39.0000,0.0,Compact Cars,False,-1,NaN,NaN,23,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013
43493,Subaru,Legacy AWD,1993,Y,23.0000,0.0,34.0000,0.0,Compact Cars,False,-1,NaN,NaN,21,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013
43494,Subaru,Legacy AWD,1993,Y,23.0000,0.0,34.0000,0.0,Compact Cars,False,-1,NaN,NaN,21,0.0,0,0.0,0.0,0.0,0.0,0,0,0,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013


In [100]:
for i in mpg_df.items:
    print(i)
#     print(f'{v['make']}+{v['model']}+{v['year']})
    

TypeError: 'method' object is not iterable

In [ ]:
fuel_econ = pd.DataFrame()

# EDA

The dataset comes in the form o

# References & Sources

Countries and states setting target years to ban sale of fossil fuel powered vehicles:  
> Reuters, 11/18/2020 — [Fossil fuel-based vehicle bans across the world](https://news.trust.org/item/20201118095737-8h1uh)  
>  
> Reuters, 11/8/2018 — [Germany City Specific Diesel Bans](https://www.reuters.com/article/us-germany-emissions-factbox/factbox-german-cities-ban-older-diesel-cars-idUSKBN1ND1ZV?edition-redirect=in)  
>  
> Insider, 1/12/2019 — [Cities Around the World Enact Plans to Reduce Emissions through Vehicle Restrictions](https://www.businessinsider.com/cities-going-car-free-ban-2018-12#in-paris-the-first-sunday-of-every-month-is-free-of-cars-2)

Fuel economy data sourced from:  
> https://www.fueleconomy.gov/feg/download.shtml

### Data Description

- vehicle
- **atvtype** - type of alternative fuel or advanced technology vehicle
- **barrels08** - annual petroleum consumption in barrels for fuelType1 (1)
- **barrelsA08** - annual petroleum consumption in barrels for fuelType2 (1)10/10/2018 Fuel Economy Web Services
- **https**://www.fueleconomy.gov/feg/ws/index.shtml 3/7
- **charge120** - time to charge an electric vehicle in hours at 120 V
- **charge240** - time to charge an electric vehicle in hours at 240 V
- **city08** - city MPG for fuelType1 (2), (11)
- **city08U** - unrounded city MPG for fuelType1 (2), (3)
- **cityA08** - city MPG for fuelType2 (2)
- **cityA08U** - unrounded city MPG for fuelType2 (2), (3)
- **cityCD** - city gasoline consumption (gallons/100 miles) in charge depleting mode (4)
- **cityE** - city electricity consumption in kw-hrs/100 miles
- **cityUF** - EPA city utility factor (share of electricity) for PHEV
- **co2** - tailpipe CO2 in grams/mile for fuelType1 (5)
- **co2A** - tailpipe CO2 in grams/mile for fuelType2 (5)
- **co2TailpipeAGpm** - tailpipe CO2 in grams/mile for fuelType2 (5)
- **co2TailpipeGpm**- tailpipe CO2 in grams/mile for fuelType1 (5)
- **comb08** - combined MPG for fuelType1 (2), (11)
- **comb08U** - unrounded combined MPG for fuelType1 (2), (3)
- **combA08** - combined MPG for fuelType2 (2)
- **combA08U** - unrounded combined MPG for fuelType2 (2), (3)
- **combE** - combined electricity consumption in kw-hrs/100 miles
- **combinedCD** - combined gasoline consumption (gallons/100 miles) in charge depleting mode (4)
- **combinedUF** - EPA combined utility factor (share of electricity) for PHEV
- **cylinders** - engine cylinders
- **displ** - engine displacement in liters
- **drive** - drive axle type
- **emissionsList**
- **engId** - EPA model type index
- **eng_dscr** - engine descriptor; see http://www.fueleconomy.gov/feg/findacarhelp.shtml#engine
- **evMotor** - electric motor (kw-hrs)
- **feScore** - EPA Fuel Economy Score (-1 = Not available)
- **fuelCost08** - annual fuel cost for fuelType1 ($\$$) (7)
- **fuelCostA08** - annual fuel cost for fuelType2 ($\$$) (7)
- **fuelType** - fuel type with fuelType1 and fuelType2 (if applicable)
- **fuelType1** - fuel type 1. For single fuel vehicles, this will be the only fuel. For dual fuel vehicles, this
- **will** be the conventional fuel.
- **fuelType2** - fuel type 2. For dual fuel vehicles, this will be the alternative fuel (e.g. E85, Electricity,
- **CNG**, LPG). For single fuel vehicles, this field is not used
- **ghgScore** - EPA GHG score (-1 = Not available)10/10/2018 Fuel Economy Web Services
- **ghgScoreA** - EPA GHG score for dual fuel vehicle running on the alternative fuel (-1 = Not available)
- **guzzler**- if G or T, this vehicle is subject to the gas guzzler tax
- **highway08** - highway MPG for fuelType1 (2), (11)
- **highway08U** - unrounded highway MPG for fuelType1 (2), (3)
- **highwayA08** - highway MPG for fuelType2 (2)
- **highwayA08U** - unrounded highway MPG for fuelType2 (2),(3)
- **highwayCD** - highway gasoline consumption (gallons/100miles) in charge depleting mode (4)
- **highwayE** - highway electricity consumption in kw-hrs/100 miles
- **highwayUF** - EPA highway utility factor (share of electricity) for PHEV
- **hlv** - hatchback luggage volume (cubic feet) (8)
- **hpv** - hatchback passenger volume (cubic feet) (8)
- **id** - vehicle record id
- **lv2** - 2 door luggage volume (cubic feet) (8)
- **lv4** - 4 door luggage volume (cubic feet) (8)
- **make** - manufacturer (division)
- **mfrCode** - 3-character manufacturer code
- **model** - model name (carline)
- **mpgData** - has My MPG data; see yourMpgVehicle and yourMpgDriverVehicle
- **phevBlended** - if true, this vehicle operates on a blend of gasoline and electricity in charge depleting
- **mode**
- **pv2** - 2-door passenger volume (cubic feet) (8)
- **pv4** - 4-door passenger volume (cubic feet) (8)
- **rangeA** - EPA range for fuelType2
- **rangeCityA** - EPA city range for fuelType2
- **rangeHwyA** - EPA highway range for fuelType2
- **trans_dscr** - transmission descriptor; see http://www.fueleconomy.gov/feg/findacarhelp.shtml#trany
- **trany** - transmission
- **UCity** - unadjusted city MPG for fuelType1; see the description of the EPA test procedures
- **UCityA** - unadjusted city MPG for fuelType2; see the description of the EPA test procedures
- **UHighway** - unadjusted highway MPG for fuelType1; see the description of the EPA test procedures
- **UHighwayA** - unadjusted highway MPG for fuelType2; see the description of the EPA test procedures
- **VClass** - EPA vehicle size class
- **year** - model year
- **youSaveSpend** - you save/spend over 5 years compared to an average car ($\$$). Savings are positive; a
- **greater** amount spent yields a negative number. For dual fuel vehicles, this is the cost savings for
- **gasoline**
- **sCharger** - if S, this vehicle is supercharged10/10/2018 Fuel Economy Web Services
- **tCharger** - if T, this vehicle is turbocharged
- **c240Dscr** - electric vehicle charger description
- **charge240b** - time to charge an electric vehicle in hours at 240 V using the alternate charger
- **c240bDscr** - electric vehicle alternate charger description
- **createdOn** - date the vehicle record was created (ISO 8601 format)
- **modifiedOn** - date the vehicle record was last modified (ISO 8601 format)
- **startStop** - vehicle has start-stop technology (Y, N, or blank for older vehicles)
- **phevCity** - EPA composite gasoline-electricity city MPGe for plug-in hybrid vehicles
- **phevHwy** - EPA composite gasoline-electricity highway MPGe for plug-in hybrid vehicles
- **phevComb** - EPA composite gasoline-electricity combined city-highway MPGe for plug-in hybrid
- **vehicles**